In [ ]:
%pip install gdown
import gdown

url = "https://drive.google.com/drive/folders/1Fl3JRqZrW4n-FOjstaIJmu02FkkNgpkw?usp=sharing"
gdown.download_folder(url, quiet=True)

In [ ]:
!pip install tqdm
!pip install dlib

In [ ]:
import os
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import numpy as np
import shutil
import zipfile
import dlib

shape_predictor_path = "/content/VGGFaces2/shape_predictor_5_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor_path)

def shape_to_normal(shape):
  shape_normal = []
  # num_parts = 5
  # shape[0] = right eye, right corner
  # shape[1] = right eye, left corner
  # shape[2] = left eye, left corner
  # shape[3] = left eye, right corner
  # shape[4] = nose
  for i in range(0, shape.num_parts):
    shape_normal.append((i, (shape.part(i).x, shape.part(i).y)))
  return shape_normal

def get_eyes_nose_dlib(shape):
  nose = shape[4][1]
  left_eye_x = int(shape[3][1][0] + shape[2][1][0]) // 2
  left_eye_y = int(shape[3][1][1] + shape[2][1][1]) // 2
  right_eyes_x = int(shape[1][1][0] + shape[0][1][0]) // 2
  right_eyes_y = int(shape[1][1][1] + shape[0][1][1]) // 2
  return nose, (left_eye_x, left_eye_y), (right_eyes_x, right_eyes_y)

def distance(a, b):
    return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def cosine_formula(length_line1, length_line2, length_line3):
    cos_a = -(length_line3 ** 2 - length_line2 ** 2 - length_line1 ** 2) / (2 * length_line2 * length_line1)
    return cos_a

def rotate_point(origin, point, angle):
    ox, oy = origin
    px, py = point

    qx = ox + np.cos(angle) * (px - ox) - np.sin(angle) * (py - oy)
    qy = oy + np.sin(angle) * (px - ox) + np.cos(angle) * (py - oy)
    return qx, qy

def is_between(point1, point2, point3, extra_point):
    c1 = (point2[0] - point1[0]) * (extra_point[1] - point1[1]) - (point2[1] - point1[1]) * (extra_point[0] - point1[0])
    c2 = (point3[0] - point2[0]) * (extra_point[1] - point2[1]) - (point3[1] - point2[1]) * (extra_point[0] - point2[0])
    c3 = (point1[0] - point3[0]) * (extra_point[1] - point3[1]) - (point1[1] - point3[1]) * (extra_point[0] - point3[0])
    if (c1 < 0 and c2 < 0 and c3 < 0) or (c1 > 0 and c2 > 0 and c3 > 0):
        return True
    else:
        return False
def align_face(image):
  gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  faces = detector(gray, 0)
  if len(faces) > 0:
    for face in faces:
      # Extract face bounding box coordinates
      x, y, w, h, shape = face.left(), face.top(), face.width(), face.height(), predictor(gray, face)

      # Get facial landmarks for the detected face
      shape = shape_to_normal(shape)

      # Get the nose, left eye, and right eye positions
      nose, left_eye, right_eye = get_eyes_nose_dlib(shape)

      # Calculate the center of the forehead and the predicted center of the face
      center_of_forehead = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
      center_pred = (int((x + w) / 2), int((y + y) / 2))

      # Calculate the distances for cosine law
      length_line1 = distance(center_of_forehead, nose)
      length_line2 = distance(center_pred, nose)
      length_line3 = distance(center_pred, center_of_forehead)

      # Calculate the angle for alignment
      cos_a = cosine_formula(length_line1, length_line2, length_line3)
      angle = np.arccos(cos_a)

      # Rotate the face based on the calculated angle
      rotated_point = rotate_point(nose, center_of_forehead, angle)
      rotated_point = (int(rotated_point[0]), int(rotated_point[1]))

      # Adjust the angle based on direction (if needed)
      if is_between(nose, center_of_forehead, center_pred, rotated_point):
          angle = np.degrees(-angle)
      else:
          angle = np.degrees(angle)

      # Convert the image to PIL format for rotation and then back to NumPy array
      image = Image.fromarray(image)
      image = np.array(image.rotate(angle))

  return image

def main():
  # folder containing the zip files and the one that will contains the images extracted from those files
  folder_path = "/content/VGGFaces2"
  for file_name in os.listdir(folder_path):

    # zipped files in the folder biometric_systems_dataset
    if file_name.endswith(".zip"):

      # full path to the zip file
      file_path = os.path.join(folder_path, file_name)

      # create folder in the same path with the name of the zip file
      extract_folder = os.path.join(folder_path, file_name.replace(".zip", ""))
      os.makedirs(extract_folder, exist_ok=True)

      # open and extract the file
      with zipfile.ZipFile(file_path, mode="r") as zip_ref:
        zip_ref.extractall(extract_folder)

        for root, _, files in os.walk(extract_folder):
          for image_file in files:
            if image_file.lower().endswith('.jpg'):
              source_file = os.path.join(root, image_file)
              destination_file = os.path.join(folder_path, image_file)

              # move the image file to the destination path
              shutil.move(source_file, destination_file)

        # remove the extracted folder and all its contents
        shutil.rmtree(extract_folder)
        os.remove(extract_folder + ".zip")

  image_path = "/content/VGGFaces2/" + os.listdir(folder_path)[0]
  image = cv2.imread(image_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  aligned_face = align_face(image)
  plt.imshow(image)
  plt.show()
  plt.imshow(aligned_face)
  plt.show()

if __name__ == "__main__":
  main()